In [1]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read in data
trial_data = pd.read_csv("Resources/clinicaltrial_data.csv")
mouse_data = pd.read_csv("Resources/mouse_drug_data.csv")

# Combine the data into a single dataset
combined_data = pd.merge(trial_data, mouse_data, on='Mouse ID', how="outer")

# Display the data table for preview
combined_data.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug
0,b128,0,45.000000,0,Capomulin
1,b128,5,45.651331,0,Capomulin
2,b128,10,43.270852,0,Capomulin
3,b128,15,43.784893,0,Capomulin
4,b128,20,42.731552,0,Capomulin


## Tumor Response to Treatment

In [5]:
# Store the Mean Tumor Volume Data Grouped by Drug and Timepoint 
mean_grouped_data = combined_data.groupby(['Drug', 'Timepoint']).mean()
mean_grouped_data

Tumor Volume (mm3)  Metastatic Sites
Drug      Timepoint                                      
Capomulin 0                   45.000000          0.000000
          5                   44.266086          0.160000
          10                  43.084291          0.320000
          15                  42.064317          0.375000
          20                  40.716325          0.652174
          25                  39.939528          0.818182
          30                  38.769339          1.090909
          35                  37.816839          1.181818
          40                  36.958001          1.380952
          45                  36.236114          1.476190
Ceftamin  0                   45.000000          0.000000
          5                   46.503051          0.380952
          10                  48.285125          0.600000
          15                  50.094055          0.789474
          20                  52.157049          1.111111
          25                  54.287674          1.500000
          30                  56.769517          1.937500
          35                  58.827548          2.071429
          40                  61.467895          2.357143
          45                  64.132421          2.692308
Infubinol 0                   45.000000          0.000000
          5                   47.062001          0.280000
          10                  49.403909          0.666667
          15                  51.296397          0.904762
          20                  53.197691          1.050000
          25                  55.715252          1.277778
          30                  58.299397          1.588235
          35                  60.742461          1.666667
          40                  63.162824          2.100000
          45                  65.755562          2.111111
...                                 ...               ...
Ramicane  0                   45.000000          0.000000
          5                   43.944859          0.120000
          10                  42.531957          0.250000
          15                  41.495061          0.333333
          20                  40.238325          0.347826
          25                  38.974300          0.652174
          30                  38.703137          0.782609
          35                  37.451996          0.952381
          40                  36.574081          1.100000
          45                  34.955595          1.250000
Stelasyn  0                   45.000000          0.000000
          5                   47.527452          0.240000
          10                  49.463844          0.478261
          15                  51.529409          0.782609
          20                  54.067395          0.952381
          25                  56.166123          1.157895
          30                  59.826738          1.388889
          35                  62.440699          1.562500
          40                  65.356386          1.583333
          45                  68.438310          1.727273
Zoniferol 0                   45.000000          0.000000
          5                   46.851818          0.166667
          10                  48.689881          0.500000
          15                  50.779059          0.809524
          20                  53.170334          1.294118
          25                  55.432935          1.687500
          30                  57.713531          1.933333
          35                  60.089372          2.285714
          40                  62.916692          2.785714
          45                  65.960888          3.071429

[100 rows x 2 columns]

In [12]:
# Reset Index
mean_tumor_vol = mean_grouped_data.reset_index()

# Limit data frame to Tumor Volume
mean_tumor_vol = mean_tumor_vol.drop(columns=["Metastatic Sites"])
mean_tumor_vol.head()

,Drug,Timepoint,Tumor Volume (mm3)
0,Capomulin,0,45.000000
1,Capomulin,5,44.266086
2,Capomulin,10,43.084291
3,Capomulin,15,42.064317
4,Capomulin,20,40.716325


In [ ]:
# Convert to DataFrame
mean_tumor_vol = mean_grouped_data.drop(columns='Metastatic Sites')
mean_tumor_vol_reset_index = mean_tumor_vol.reset_index()

mean_tumor_df = pd.DataFrame({'Drug':mean_tumor_vol_reset_index['Drug'],
                         'Timepoint':mean_tumor_vol_reset_index['Timepoint'],
                         'Mean Tumor Volume (mm3)': mean_tumor_vol_reset_index['Tumor Volume (mm3)']})

# Preview DataFrame
mean_tumor_df

In [ ]:
# Store the Standard Error of Tumor Volumes Grouped by Drug and Timepoint
import scipy.stats
from scipy.stats import sem

standard_error = grouped_data['Tumor Volume (mm3)'].sem()

# Convert to DataFrame
standard_error_reset_index = standard_error.reset_index()

stderr_tumor_df = pd.DataFrame({'Drug':standard_error_reset_index['Drug'],
                         'Timepoint':standard_error_reset_index['Timepoint'],
                         'SEM Tumor Volume (mm3)': standard_error_reset_index['Tumor Volume (mm3)']})


# Preview DataFrame
stderr_tumor_df.head(10)

In [ ]:
# Minor Data Munging to Re-Format the Data Frames
mean_tumor_df_reformat = mean_tumor_df.pivot(index="Timepoint", columns="Drug")

# Preview that Reformatting worked
mean_tumor_df_reformat

In [ ]:
# x values
timepoints = np.arange(0, 50, 5)

# Capomulin: y and y-error
cap = mean_tumor_df_reformat.iloc[:,0]
cap_y = stderr_tumor_df.iloc[0:10,2]

# Infubinol: y and y-error
infu = mean_tumor_df_reformat.iloc[:,2]
infu_y = stderr_tumor_df.iloc[20:30,2]

# Ketapril: y and y-error
keta = mean_tumor_df_reformat.iloc[:,3]
keta_y = stderr_tumor_df.iloc[30:40,2]


# Placebo: y and y-error
plac = mean_tumor_df_reformat.iloc[:,5]
plac_y = stderr_tumor_df.iloc[50:60,2]


# Plot
plt.errorbar(timepoints, cap, yerr=cap_y, fmt='o', color='r', ecolor='r', label="Capomulin", linestyle='--', linewidth=0.5)
plt.errorbar(timepoints, infu, yerr=infu_y, fmt='^', color='b', ecolor='b', label="Infubinol", linestyle='--', linewidth=0.5)
plt.errorbar(timepoints, keta, yerr=keta_y, fmt='s', color='g', ecolor='g', label="Ketapril", linestyle='--', linewidth=0.5)
plt.errorbar(timepoints, plac, yerr=plac_y, fmt='d', color='k', ecolor='k', label="Placebo", linestyle='--', linewidth=0.5)

#Format
plt.title("Tumor Response to Treatment")
plt.xlabel("Time (Days)")
plt.ylabel("Average Tumor Volume (mm3)")
plt.legend()
plt.grid()
plt.show()

## Metastatic Response to Treatment

In [ ]:
# Store the Mean Metastatic Site Data Grouped by Drug and Timepoint 
mean_met_site = mean_grouped_data.drop(columns='Tumor Volume (mm3)')

# Convert to DataFrame
mean_met_site_reset_index = mean_met_site.reset_index()

mean_met_df = pd.DataFrame({'Drug':mean_met_site_reset_index['Drug'],
                         'Timepoint':mean_met_site_reset_index['Timepoint'],
                         'Mean Metastatic Sites': mean_met_site_reset_index['Metastatic Sites']})

# Preview DataFrame
mean_met_df.head()

In [ ]:
# Store the Standard Error associated with Metastatic Sites Grouped by Drug and Timepoint 
standard_error = grouped_data['Metastatic Sites'].sem()

# Convert to DataFrame
standard_error_reset_index = standard_error.reset_index()

stderr_met_df = pd.DataFrame({'Drug':standard_error_reset_index['Drug'],
                         'Timepoint':standard_error_reset_index['Timepoint'],
                         'SEM Metastatic Sites': standard_error_reset_index['Metastatic Sites']})

# Preview DataFrame
stderr_met_df.head()

In [ ]:
# Minor Data Munging to Re-Format the Data Frames
mean_met_df_reformat = mean_met_df.pivot(index="Timepoint", columns="Drug")

# Preview that Reformatting worked
mean_met_df_reformat

In [ ]:
# Capomulin: y and y-error
c_met = mean_met_df_reformat.iloc[:,0]
c_met_y = stderr_met_df.iloc[0:10,2]

# Infubinol: y and y-error
i_met = mean_met_df_reformat.iloc[:,2]
i_met_y = stderr_met_df.iloc[20:30,2]

# Ketapril: y and y-error
k_met = mean_met_df_reformat.iloc[:,3]
k_met_y = stderr_met_df.iloc[30:40,2]


# Placebo: y and y-error
p_met = mean_met_df_reformat.iloc[:,5]
p_met_y = stderr_met_df.iloc[50:60,2]


# Plot
plt.errorbar(timepoints, c_met, yerr=c_met_y, fmt='o', color='r', ecolor='r', label="Capomulin", linestyle='--', linewidth=0.5)
plt.errorbar(timepoints, i_met, yerr=i_met_y, fmt='^', color='b', ecolor='b', label="Infubinol", linestyle='--', linewidth=0.5)
plt.errorbar(timepoints, k_met, yerr=k_met_y, fmt='s', color='g', ecolor='g', label="Ketapril", linestyle='--', linewidth=0.5)
plt.errorbar(timepoints, p_met, yerr=p_met_y, fmt='d', color='k', ecolor='k', label="Placebo", linestyle='--', linewidth=0.5)

#Format
plt.title("Metastatic Spread During Treatment")
plt.xlabel("Time (Days)")
plt.ylabel("Metastatic Sites")
plt.legend()
plt.grid()
plt.show()

## Survival Rates

In [ ]:
# Store the Count of Mice Grouped by Drug and Timepoint (W can pass any metric)
grouped_mice = combined_data.groupby(['Drug', 'Timepoint'])
mouse_ct = grouped_mice['Mouse ID'].count()

# Convert to DataFrame
mouse_ct_reset = mouse_ct.reset_index()
mouse_ct_df = pd.DataFrame(mouse_ct_reset)
mouse_ct_df = mouse_ct_df.rename(columns={"Mouse ID":"Mouse Count"})

# Preview DataFrame
mouse_ct_df.head()

In [ ]:
# Minor Data Munging to Re-Format the Data Frames
mouse_ct_df_reformat = mouse_ct_df.pivot(index="Timepoint", columns="Drug")

# Preview the Data Frame
mouse_ct_df_reformat

In [ ]:
# y-values
c_survival_rate = mouse_ct_df_reformat.iloc[:,0]/mouse_ct_df_reformat.iloc[0,0] * 100
i_survival_rate = mouse_ct_df_reformat.iloc[:,2]/mouse_ct_df_reformat.iloc[0,0] * 100
k_survival_rate = mouse_ct_df_reformat.iloc[:,3]/mouse_ct_df_reformat.iloc[0,0] * 100
p_survival_rate = mouse_ct_df_reformat.iloc[:,5]/mouse_ct_df_reformat.iloc[0,0] * 100

# Plot
plt.plot(timepoints, c_survival_rate, '--o', c='r', label='Capomulin', linewidth=0.5)
plt.plot(timepoints, i_survival_rate, '--^', c='b', label='Infubinol', linewidth=0.5)
plt.plot(timepoints, k_survival_rate, '--s', c='g', label='Ketapril', linewidth=0.5)
plt.plot(timepoints, p_survival_rate, '--d', c='k', label='Placebo', linewidth=0.5)

plt.title('Survival During Treatment')
plt.xlabel('Time (Days)')
plt.ylabel('Survival Rate (%)')
plt.legend()
plt.grid()
plt.show()

## Summary Bar Graph

In [ ]:
# Calculate the percent changes of tumor volume for each drug
first_timepoint = mean_tumor_df_reformat.iloc[0,:]
last_timepoint = mean_tumor_df_reformat.iloc[-1,:]

pct_change = ((last_timepoint - first_timepoint) / first_timepoint) * 100
pct_change

In [ ]:
# Store all Relevant Percent Changes into a Tuple


# Splice the data between passing and failing drugs


# Orient widths. Add labels, tick marks, etc. 


# Use functions to label the percentages of changes


# Call functions to implement the function calls


# Save the Figure


# Show the Figure
fig.show()